In [1]:
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

In [9]:
def reshape_input(row):
    reshaped = np.zeros((6, 10))
    point_index = 0
    for i in range(6):
        num_points = 5 + i
        reshaped[i, :num_points] = row[point_index:point_index + num_points]
        point_index += num_points
    return reshaped

In [11]:
from sklearn.model_selection import train_test_split

input_df = pd.read_csv('dataset/field_data.csv', header=None)
output_df = pd.read_csv('dataset/control_voltage.csv', header=None)

full_df = pd.concat([input_df, output_df], axis=1)

train_df, test_df = train_test_split(full_df, test_size=0.1, random_state=1337)

train_df.to_csv('dataset/trainset.csv', index=False, header=False)
test_df.to_csv('dataset/testset.csv', index=False, header=False)


In [2]:
class mlp(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim):
        super(mlp, self).__init__()
        self.network = nn.Sequential(
            nn.Linear(input_dim, hidden_dim1),
            nn.ReLU(),
            nn.Linear(hidden_dim1, hidden_dim2),
            nn.ReLU(),
            nn.Linear(hidden_dim2, output_dim)
        )

    def forward(self, x):
        return self.network(x)

In [66]:
class cnn(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(cnn, self).__init__()
        self.conv1 = nn.Conv2d(input_dim,32,kernel_size=3)
        self.conv2 = nn.Conv2d(32,64,kernel_size=3)
#         self.conv3 = nn.Conv2d(64,128,kernel_size=3,padding=1)
        self.fc1 = nn.Linear(64*2*6,512)
        self.fc2 = nn.Linear(512,256)
        self.fc3 = nn.Linear(256,output_dim)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
#         x = F.relu(self.conv3(x))
        x = x.view(x.size(0),-1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return x

In [10]:
class rnn(nn.Module):
    def __init__(self, input_dim, hidden_dim1, hidden_dim2, output_dim, num_layers=1):
        super(rnn, self).__init__()
        self.rnn_h = nn.RNN(input_dim, hidden_dim1, num_layers, nonlinearity='relu', batch_first=True, bidirectional=True)
        #self.fc1 = nn.Linear(24, 18)
        #self.fc2 = nn.Linear(18, 8)
        self.fc3 = nn.Linear(48, output_dim)

    def forward(self, x):
        out, _ = self.rnn_h(x)
        #out = out.squeeze()
        #out = self.fc1(out)
        #out = self.fc2(out)
        out = out.reshape(out.size(dim=0),out.size(dim=1)*out.size(dim=2))
        out = self.fc3(out)
        return out

In [11]:
class TrxEncoder(nn.Module):
    def __init__(self, d_token, n_head, dim_ffn):
        super(TrxEncoder, self).__init__()
        self.encoderLayer = nn.TransformerEncoderLayer(d_token, n_head, dim_ffn, batch_first=True)
        self.encoder = nn.TransformerEncoder(self.encoderLayer, num_layers=3, enable_nested_tensor=False)
        self.ffn = nn.Linear(24,2)
    
    def forward(self, x):
        x = x.reshape(len(x),24,1)
        y = self.encoder(x).squeeze()
        y = self.ffn(y)
        return y

In [61]:
class CNNdataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file,header=None)
        N = len(df)
        self.output = torch.tensor(df.iloc[:, -24:].values).float()
        input = df.iloc[:, :-24]
        input = np.array(input.apply(reshape_input, axis=1).tolist())
        input = torch.tensor(input).float().reshape(-1, 1, 6, 10)
        self.input = input - 10
        #self.output = -self.output
    
    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.input[idx], self.output[idx]

In [3]:
class dataset(Dataset):
    def __init__(self, csv_file):
        df = pd.read_csv(csv_file,header=None)
        N = len(df)
        self.input = torch.tensor(df.iloc[:, :-2].values).float()
        self.input = self.input - 10
        self.output = torch.tensor(df.iloc[:, -2:].values).float()
        #self.output = -self.output
    
    def __len__(self):
        return len(self.output)

    def __getitem__(self, idx):
        return self.input[idx], self.output[idx]

In [4]:
def train(data_loader, model, optimizer, epoch, criterion, device):
    # set the model to training mode
    model.train()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        optimizer.zero_grad()
        pred = model(data)
        loss = criterion(pred, target)
        loss.backward()
        optimizer.step()
        
    return loss.item()

In [5]:
def test(data_loader, model, criterion, device):
    # set the model to evaluation mode
    model.eval()
    
    for (step, value) in enumerate(data_loader):
        data = value[0].to(device)
        target = value[1].to(device)
        pred = model(data)
        loss = criterion(pred, target)
        
    return loss.item()

In [67]:
# Check if CUDA is available and set device to GPU if it is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)

# train_set = dataset("trainset.csv")
# test_set = dataset("testset.csv")

train_set = CNNdataset("dataset/trainset.csv")
test_set = CNNdataset("dataset/testset.csv")

input_dim = len(train_set[0][0])
output_dim = len(test_set[0][1])
hidden_dim1 = 18
hidden_dim2 = 8
N = train_set.__len__()

# setup hyperparameters
batch_size = 512
lr = 1e-3
n_epoch = 1000

train_loader = DataLoader(
                train_set,
                batch_size=batch_size
            )
test_loader = DataLoader(
                test_set,
                batch_size=batch_size
            )

d_token = 1
n_head = 1
dim_ffn = 24
# model = mlp(input_dim, hidden_dim1, hidden_dim2, output_dim)
model = cnn(1,output_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # Use Adam optimizer
criterion = nn.MSELoss()

# training and evaluation
for ep in range(n_epoch+1):
    train_loss = train(train_loader, model, optimizer, n_epoch, criterion, device)
    if ep%100 == 0:
        test_loss = test(test_loader, model, criterion, device)
        print(f"Epoch {ep}, train_loss: {train_loss}, test_loss: {test_loss}")

Epoch 0, train_loss: 49.61119842529297, test_loss: 56.10733413696289
Epoch 100, train_loss: 14.16224193572998, test_loss: 15.906962394714355
Epoch 200, train_loss: 13.33887004852295, test_loss: 15.377961158752441
Epoch 300, train_loss: 4.131800174713135, test_loss: 4.427380561828613
Epoch 400, train_loss: 3.569132089614868, test_loss: 4.347091197967529
Epoch 500, train_loss: 3.4396004676818848, test_loss: 4.194437026977539
Epoch 600, train_loss: 3.4713988304138184, test_loss: 4.480471134185791
Epoch 700, train_loss: 2.9841606616973877, test_loss: 4.277564525604248
Epoch 800, train_loss: 2.940187931060791, test_loss: 4.467309951782227
Epoch 900, train_loss: 2.816152334213257, test_loss: 4.644703388214111
Epoch 1000, train_loss: 2.7179808616638184, test_loss: 4.536198616027832


In [ ]:
# Check if CUDA is available and set device to GPU if it is
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.manual_seed(1337)
torch.cuda.manual_seed(1337)

train_set = dataset("dataset/trainset.csv")
test_set = dataset("dataset/testset.csv")

# train_set = CNNdataset("dataset/trainset.csv")
# test_set = CNNdataset("dataset/testset.csv")

input_dim = len(train_set[0][0])
output_dim = len(test_set[0][1])
hidden_dim1 = 18
hidden_dim2 = 8
N = train_set.__len__()

# setup hyperparameters
batch_size = 512
lr = 1e-2
n_epoch = 1000

train_loader = DataLoader(
                train_set,
                batch_size=batch_size
            )
test_loader = DataLoader(
                test_set,
                batch_size=batch_size
            )

d_token = 1
n_head = 1
dim_ffn = 24
model = mlp(input_dim, hidden_dim1, hidden_dim2, output_dim)
# model = cnn(1,output_dim)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)   # Use Adam optimizer
criterion = nn.MSELoss()

# training and evaluation
for ep in range(n_epoch+1):
    train_loss = train(train_loader, model, optimizer, n_epoch, criterion, device)
    if ep%100 == 0:
        test_loss = test(test_loader, model, criterion, device)
        print(f"Epoch {ep}, train_loss: {train_loss}, test_loss: {test_loss}")

In [68]:
x,y = test_set[:]
x = x.to(device)
y = y.to(device)
print(x)

tensor([[[[-64.4572, -59.3870, -64.2015,  ..., -10.0000, -10.0000, -10.0000],
          [-63.1429, -51.6804, -66.9819,  ..., -10.0000, -10.0000, -10.0000],
          [-60.2925, -49.8265, -54.4100,  ..., -10.0000, -10.0000, -10.0000],
          [-59.8568, -62.4871, -78.1282,  ..., -49.5859, -10.0000, -10.0000],
          [-54.9118, -69.3942, -64.1117,  ..., -53.5086, -63.3074, -10.0000],
          [-62.0897, -53.7024, -54.6587,  ..., -75.1512, -60.4022, -60.4174]]],


        [[[-62.1884, -62.7863, -63.5164,  ..., -10.0000, -10.0000, -10.0000],
          [-62.1999, -53.0958, -60.5067,  ..., -10.0000, -10.0000, -10.0000],
          [-59.3836, -51.2216, -57.0360,  ..., -10.0000, -10.0000, -10.0000],
          [-63.0597, -64.4534, -69.9967,  ..., -50.2700, -10.0000, -10.0000],
          [-57.3588, -80.7438, -66.5682,  ..., -54.9454, -64.6188, -10.0000],
          [-63.8788, -55.8545, -54.6261,  ..., -73.7990, -63.4980, -58.6775]]],


        [[[-60.5257, -58.1082, -64.3575,  ..., -10.0000,

In [69]:
pred = model(x)
print(f"Shape: {pred.shape}")
print(pred[0:10])

Shape: torch.Size([1000, 24])
tensor([[13.4066, 14.2216, 14.8801, 15.1407, 14.2608, 12.3907, 11.1114, 10.1793,
          9.8868, 10.8813, 12.2808, 14.0165, 14.8471, 15.4980, 17.2697, 18.1930,
         17.9698, 16.7802, 15.2887, 14.8395, 15.0425, 15.2330, 14.6985, 13.5115],
        [ 9.9945, 10.5698, 11.1217, 12.1377, 12.7065, 13.8208, 16.4646, 18.6257,
         18.9761, 17.9130, 16.8964, 17.1658, 17.5228, 16.7609, 16.7364, 16.2897,
         15.4717, 13.4701, 11.0119,  9.6383,  9.0897,  8.2071,  6.7179,  5.8105],
        [14.9793, 16.2931, 17.2922, 17.8289, 16.9360, 14.7303, 13.3557, 12.5745,
         12.8002, 13.8111, 15.1972, 15.7603, 15.8332, 15.9810, 16.8784, 17.2648,
         17.0788, 17.2077, 17.4300, 16.7571, 15.8545, 14.5758, 13.3823, 12.7387],
        [ 4.0901,  3.8001,  3.3824,  3.1120,  2.8654,  2.6301,  2.6712,  2.6645,
          2.4763,  2.5490,  2.4817,  2.4717,  2.3768,  2.3895,  2.5267,  2.5085,
          2.4353,  2.1888,  2.1473,  2.4604,  3.0730,  3.4398,  3.6733,  3.9

In [70]:
from sklearn import metrics

# Measure RMSE error.  RMSE is common for regression.

score = np.sqrt(metrics.mean_squared_error(pred.cpu().detach(), y.cpu().detach()))
print(f"Final score (RMSE): {score}")

score = torch.sqrt(torch.nn.functional.mse_loss(pred, y))
print(f"Final score (RMSE) with training dataset: {score}")

Final score (RMSE): 2.1971957683563232
Final score (RMSE) with training dataset: 2.197195529937744


In [71]:
# Sample predictions
for i in range(50):
    print(f"{i+1}. Actual: {y[i]}, " + f"Predicted: {pred[i]}")

1. Actual: tensor([11.9500, 11.0703, 14.0351, 16.5361, 13.7649, 13.6010, 13.0974, 11.0100,
        10.3430,  9.8325, 11.8579, 14.2976, 15.9465, 16.1321, 18.2178, 19.0000,
        18.7118, 19.0000, 17.5798, 14.6208, 14.1341, 15.1884, 16.8914, 17.4756],
       device='cuda:0'), Predicted: tensor([13.4066, 14.2216, 14.8801, 15.1407, 14.2608, 12.3907, 11.1114, 10.1793,
         9.8868, 10.8813, 12.2808, 14.0165, 14.8471, 15.4980, 17.2697, 18.1930,
        17.9698, 16.7802, 15.2887, 14.8395, 15.0425, 15.2330, 14.6985, 13.5115],
       device='cuda:0', grad_fn=<SelectBackward0>)
2. Actual: tensor([12.4021, 10.4130, 10.0268, 11.5515, 14.5753, 14.2685, 15.0475, 18.1019,
        19.0000, 19.0000, 16.7002, 18.0079, 19.0000, 16.8844, 16.7672, 16.6725,
        15.0931, 12.7830, 11.1938,  9.3530,  7.7586,  5.1286,  7.6062,  5.9287],
       device='cuda:0'), Predicted: tensor([ 9.9945, 10.5698, 11.1217, 12.1377, 12.7065, 13.8208, 16.4646, 18.6257,
        18.9761, 17.9130, 16.8964, 17.1658, 17.5228,

In [ ]:
train_set[0]

In [ ]:
x,y = train_set[range(4)]
x = x.reshape([4,12,2])
rnn = nn.RNN(2, 1, 1, nonlinearity='relu', batch_first=True)

In [ ]:
print(x)

In [ ]:
y = rnn(x)
print(y)

In [78]:
x = torch.ones([16,24,1])
w = torch.ones([24,18])
z = torch.matmul(torch.t(w),x)
print(z.shape)

torch.Size([16, 18, 1])
